In [62]:
# Import all dependencies
import numpy as np
import pandas as pd
#In order to show all columns available
pd.set_option('display.max_columns', 200)

#Sklearn imports
from sklearn.preprocessing import LabelEncoder, Imputer

#file system management
import os

#Graphing libs
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
# Load in ALL dataframes

apptrain = pd.read_csv('./Dataset/application_train.csv').sort_values('SK_ID_CURR')#.reset_index(drop = True).loc[:1000, :]
apptest = pd.read_csv('./Dataset/application_test.csv').sort_values('SK_ID_CURR')#.reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('./Dataset/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU'])#.reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('./Dataset/bureau_balance.csv').sort_values('SK_ID_BUREAU')#.reset_index(drop = True).loc[:1000, :]
credit_card_balance = pd.read_csv('./Dataset/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV'])#.reset_index(drop = True).loc[:1000, :]
installment_payments = pd.read_csv('./Dataset/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV'])#.reset_index(drop = True).loc[:1000, :]
pos_cash_balance = pd.read_csv('./Dataset/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV'])#.reset_index(drop = True).loc[:1000, :]
previous_application = pd.read_csv('./Dataset/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV'])#.reset_index(drop = True).loc[:1000, :]

# Automated Feature Engineering

Since apptrain is our highest feature table, and also considering our hardware limitations, let's reduce the number of features the table has. We can do this since not all of the features given will be useful for our machine learning model

In [64]:
bureau = pd.read_csv('./Dataset/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU'])#.reset_index(drop = True).loc[:1000, :]

# Manual Feature Engineering

Here we will make Polynomial features from the ext data sources combine them with domain knowledge, afterwards we will run these features through a random forest models and extract the top 50% features

In [65]:
# Code that modifies dataframes
apptrain['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
apptest['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
apptrain['DAYS_BIRTH'] = abs(apptrain['DAYS_BIRTH'])
apptest['DAYS_BIRTH'] = abs(apptrain['DAYS_BIRTH'])

In [66]:
# Take note that DK will be understood as Domain Knowledge
# Creating train data copies
apptrain_domain = apptrain.copy()
apptest_domain = apptest.copy()

#Creating variables for train data
apptrain_domain['CREDIT_INCOME_PERCENT'] = apptrain_domain['AMT_CREDIT'] * 100 / apptrain_domain['AMT_INCOME_TOTAL']
apptrain_domain['ANNUITY_INCOME_PERCENT'] = apptrain_domain['AMT_ANNUITY'] / apptrain_domain['AMT_INCOME_TOTAL']
apptrain_domain['CREDIT_TERM'] = apptrain_domain['AMT_ANNUITY'] / apptrain_domain['AMT_CREDIT']
apptrain_domain['DAYS_EMPLOYED_PERCENT'] = apptrain_domain['DAYS_EMPLOYED'] / apptrain_domain['DAYS_BIRTH']

#Creating variables for test data
apptest_domain['CREDIT_INCOME_PERCENT'] = apptest_domain['AMT_CREDIT'] / apptest_domain['AMT_INCOME_TOTAL']
apptest_domain['ANNUITY_INCOME_PERCENT'] = apptest_domain['AMT_ANNUITY'] / apptest_domain['AMT_INCOME_TOTAL']
apptest_domain['CREDIT_TERM'] = apptest_domain['AMT_ANNUITY'] / apptest_domain['AMT_CREDIT']
apptest_domain['DAYS_EMPLOYED_PERCENT'] = apptest_domain['DAYS_EMPLOYED'] / apptest_domain['DAYS_BIRTH']

In [67]:
# Creating polynomial features starting from the dataframe we already created domain knowledge features for
poly_features = apptrain_domain[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = apptest_domain[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# Removing target column so fits are the same
poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])

# Imputing values
imputer = Imputer(strategy = 'median')
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

print('poly_features shape: {}'.format(poly_features.shape))
print('poly_features_test shape: {}'.format(poly_features_test.shape))

poly_features shape: (307511, 4)
poly_features_test shape: (48744, 4)


In [68]:
from sklearn.preprocessing import PolynomialFeatures
#Create polynomial object with specific degree
poly_transformer = PolynomialFeatures(degree = 3)
poly_transformer.fit(poly_features)

#transforming features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features Shape: ', poly_features.shape)

Polynomial Features Shape:  (307511, 35)


In [69]:
# Turning polynomial features into dataframe for both training and testing
poly_features = pd.DataFrame(poly_features, columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))
# Put target back into poly_features
poly_features['TARGET'] = poly_target

# Testing dataset
poly_features_test = pd.DataFrame(poly_features_test, columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))

In [70]:
# Creating common column to merge training dataset and polyfeatures dataset
poly_features['SK_ID_CURR'] = apptrain_domain['SK_ID_CURR']
pdk_training_df = apptrain_domain.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Doing same for test dataset
poly_features_test['SK_ID_CURR'] = apptest_domain['SK_ID_CURR']
pdk_testing_df = apptest_domain.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align that dataframes
pdk_training_df, pdk_testing_df = pdk_training_df.align(pdk_testing_df, join = 'inner', axis = 1)

print('polynomial domain knowledge training dataset: ', pdk_training_df.shape)
print('polynomial domain knowledge testing dataset: ', pdk_testing_df.shape)
pdk_train_labels = poly_features['TARGET']

polynomial domain knowledge training dataset:  (307511, 160)
polynomial domain knowledge testing dataset:  (48744, 160)


In [71]:
# Label Encode the dataset
le = LabelEncoder()
le_count = 0

for col in pdk_training_df:
    if pdk_training_df[col].dtype == 'object':
        if len(list(pdk_training_df[col].unique())) <= 2:
            # fit the column
            le.fit(pdk_training_df[col])
            # Transform the column both testing and training dfs
            pdk_training_df[col] = le.transform(pdk_training_df[col])
            pdk_testing_df[col] = le.transform(pdk_testing_df[col])
            
            le_count += 1
            

# One hot encode
pdk_training_df = pd.get_dummies(pdk_training_df)
pdk_testing_df = pd.get_dummies(pdk_testing_df)

print('Training features shape: {}'.format(pdk_training_df.shape))
print('Training features shape: {}'.format(pdk_testing_df.shape))
print('{} columns were label encoded'.format(le_count))

Training features shape: (307511, 281)
Training features shape: (48744, 278)
3 columns were label encoded


Align the datasets to equal amount of features

In [72]:
# Align dataframes
pdk_training_df, pdk_testing_df = pdk_training_df.align(pdk_testing_df, join = 'inner', axis = 1)
pdk_training_df['TARGET'] = poly_target

print('Training Features shape: ', pdk_training_df.shape)
print('Testing features shape: ', pdk_testing_df.shape)
print("We're back on track, remember the training dataset will have one column more since it DOES have the targets")

Training Features shape:  (307511, 279)
Testing features shape:  (48744, 278)
We're back on track, remember the training dataset will have one column more since it DOES have the targets


# Features so far

We've inlcuded our Domain Knowledge features and our Polynomial features, To add more features we can include other dataframes and merge them into our featured engineering dataframe

In [151]:
# Let's try to add features from bureau
bureau_copy = bureau.iloc[:1000]
bureau_copy

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.000,0.0,0.000,0.0,Consumer credit,-155,0.0
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.000,113166.0,0.000,0.0,Consumer credit,-6,4630.5
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.000,373239.0,0.000,0.0,Consumer credit,-16,10822.5
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.000,110281.5,NaN,0.0,Consumer credit,-10,9364.5
1486113,100002,6113835,Closed,currency 1,-1043,0,62.0,-967.0,0.000,0,67500.000,NaN,NaN,0.0,Credit card,-758,0.0
1337779,100002,6158903,Active,currency 1,-1042,0,780.0,NaN,NaN,0,450000.000,245781.0,0.000,0.0,Consumer credit,-7,0.0
675684,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0


## Extracting features from the bureau table

We'll add the AMT_CREDIT_SUM_DEBT to our main set of features using the following code

In [150]:
# Selecting the feature we want to extract and grouping it by main df's id's
bureau_feature = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']]
bureau_feature_groupedbyid = bureau_feature.groupby('SK_ID_CURR')

# Running aggregate function on the feature
amt_credit_sum_debt_byid = bureau_feature_groupedbyid['AMT_CREDIT_SUM_DEBT'].agg(np.mean)

# To be on the safe side we copy our dataframes
pdk_copy = pdk_training_df.copy()
pdk_test_copy = pdk_testing_df.copy()

# Clean the series change to dataframe
amt_credit_sum_debt_byid = amt_credit_sum_debt_byid.to_frame().dropna()

# the new frames should contain all the data and be the original shape +1
new_feature_frame = pdk_copy.merge(amt_credit_sum_debt_byid, right_index=True, left_on='SK_ID_CURR')
new_feature_testing_frame = pdk_test_copy.merge(amt_credit_sum_debt_byid, right_index=True, left_on='SK_ID_CURR')

print(new_feature_frame.shape)
print(new_feature_testing_frame.shape)

(256131, 280)
(41308, 279)


In [155]:
bureau_feature2 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM']]
bureau_feature2_groupedbyid = bureau_feature2.groupby('SK_ID_CURR')

amt_credit_sum_byid = bureau_feature2_groupedbyid['AMT_CREDIT_SUM'].agg(np.mean)
amt_credit_sum_byid = amt_credit_sum_byid.to_frame().dropna()

new_feature_frame = new_feature_frame.merge(amt_credit_sum_byid, right_index=True, left_on='SK_ID_CURR')
new_feature_testing_frame = new_feature_testing_frame.merge(amt_credit_sum_byid, right_index=True, left_on='SK_ID_CURR')

print(new_feature_frame.shape)
print(new_feature_testing_frame.shape)

(256130, 281)
(41307, 280)


## Extracting features from the previous application table

In [174]:
previous_copy_feature = previous_application[['SK_ID_CURR', 'AMT_DOWN_PAYMENT', 'AMT_APPLICATION', 'AMT_GOODS_PRICE']]
previous_copy_feature = previous_copy_feature.groupby('SK_ID_CURR')

previous_copy_feature = previous_copy_feature['AMT_DOWN_PAYMENT', 'AMT_APPLICATION', 'AMT_GOODS_PRICE'].agg(np.mean)

new_feature_frame = new_feature_frame.merge(previous_copy_feature, right_index=True, left_on='SK_ID_CURR')
new_feature_testing_frame = new_feature_testing_frame.merge(previous_copy_feature, right_index=True, left_on='SK_ID_CURR')

print(new_feature_frame.shape)
print(new_feature_testing_frame.shape)

(242770, 287)
(40598, 283)


## Extracting features from credit card balance table

# Merge dataframes together and delete duplicates

In [160]:
new_feature_frame_copy = new_feature_frame.copy()
df = new_feature_frame_copy.merge(pdk_copy, how='outer')